# compare ingestion of old and new backend (and it's flavors)

the idea of this notebook is to present some examples of how the backends respond, therefor I didn't use pyscicat on purpose but rather wanted to look at raw requests.

In [7]:
from pprint import pprint
from datetime import datetime
import json
import requests
from pprint import pprint


#some connection details to scicatlive instances running different backend versions
scicat_new_backend_package={
    "host": "http://35.205.92.39",
}

scicat_new_backend_swap2577={
    "host": "http://35.195.235.21",
}

scicat_new_backend_master={
    "host": "http://35.195.188.142",
}

scicat_old_backend={
    "host": "http://35.233.107.24",
}




In [8]:
#helpers

#headers for requests
headers = {
   "accept": "application/json",
   "Content-Type": "application/json"
}

USERNAME="admin"
PASSWORD="2jf70TPNZsS"

def add_token(scicat):
    response = requests.post(scicat["host"]+"/api/v3/auth/login", headers=headers, 
                             json={"username":USERNAME,"password":PASSWORD})
    resp = json.loads(response.content)
    scicat["token"]="Bearer "+resp["access_token"]
    print("host: "+ scicat["host"], "user: "+ resp["user"]["username"], "userID: "+resp["user"]["id"])
    
def add_token_old(scicat):
    response = requests.post(scicat["host"]+"/api/v3/Users/login", headers=headers, 
                             json={"username":USERNAME,"password":PASSWORD})
    resp = json.loads(response.content)
    scicat["token"]=resp["id"]
    print("host: "+ scicat["host"], "userID: "+resp["userId"])
    

# post request

def post(scicat, url, data, new_backend=True):
    if new_backend:
        response = requests.post(scicat["host"]+url, headers={**headers,"Authorization": scicat["token"],}, json=data)
    else:
        response = requests.post(scicat["host"]+url+"?access_token="+scicat["token"], headers=headers, json=data)
    return(json.loads(response.content))


# run request against different scicat instances at once
def run_all(url,data):
    print("="*20 + "scicat_new_backend_package" + "="*20)
    pprint(post(scicat_new_backend_package, url, data=data))

    print("\n"*3)

    print("="*20 + "scicat_new_backend_swap2577" + "="*20)
    pprint(post(scicat_new_backend_swap2577, url, data=data))

    print("\n"*3)

    print("="*20 + "scicat_new_backend_master" + "="*20)
    pprint(post(scicat_new_backend_master, url, data=data))

    print("\n"*3)

    print("="*20 + "scicat_old_backend" + "="*20)
    pprint(post(scicat_old_backend, url, data=data, new_backend=False))

In [9]:
#deal with authorization (add token)

add_token(scicat_new_backend_package)
add_token(scicat_new_backend_swap2577)
add_token(scicat_new_backend_master)
add_token_old(scicat_old_backend)

host: http://35.205.92.39 user: admin userID: 6368e81046013cc74b550bc1
host: http://35.195.235.21 user: admin userID: 6368ed158aaaf2ce45885798
host: http://35.195.188.142 user: admin userID: 6368fef0911030264699064e
host: http://35.233.107.24 userID: 6368e695a199ec3107c3df3f


# Dataset ingestion

Here is an example of a dataset that works for *all* available backend versions (new/old ... branches / package of new backend)

- Do we agree that all of these fields are needed ? ... different backends require different subsets of those...

In [10]:
mini_dataset ={
"ownerGroup":"mygroup",
"accessGroups":["mygroup"],
"creationLocation":"somewhere", 
"principalInvestigator":"someone", 
"datasetName":"test " + datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
"type":"raw",
"isPublished":False,   
"creationTime":str(datetime.now().isoformat()), 
"sourceFolder":"None",
"owner":"someone",
"ownerEmail":"a@b.de",
"contactEmail":"a@b.de",
"size":0,
"scientificMetadata":{"test":1.0, "test2":{"value":17,"unit":"mA"}},
"techniques": [
    {
      "pid": "string",
      "name": "string"
    }]
}

run_all("/api/v3/Datasets",mini_dataset)

====================scicat_new_backend_package====================
{'__v': 0,
 '_id': 'PID.SAMPLE.PREFIX8037fa32-f6b1-436f-b25e-5dd3e0569b2c',
 'accessGroups': ['mygroup'],
 'attachments': [],
 'contactEmail': 'a@b.de',
 'creationLocation': 'somewhere',
 'creationTime': '2022-11-08T16:06:07.814Z',
 'datablocks': [],
 'datasetName': 'test 2022-11-08-16_06_07',
 'history': [],
 'id': 'PID.SAMPLE.PREFIX8037fa32-f6b1-436f-b25e-5dd3e0569b2c',
 'isPublished': False,
 'keywords': [],
 'origdatablocks': [],
 'owner': 'someone',
 'ownerEmail': 'a@b.de',
 'ownerGroup': 'mygroup',
 'pid': 'PID.SAMPLE.PREFIX8037fa32-f6b1-436f-b25e-5dd3e0569b2c',
 'principalInvestigator': 'someone',
 'scientificMetadata': {'test': 1,
                        'test2': {'unit': 'mA',
                                  'unitSI': 'A',
                                  'value': 17,
                                  'valueSI': 0.017}},
 'sharedWith': [],
 'size': 0,
 'sourceFolder': 'None',
 'techniques': [{'_id': '636a705

In [194]:
# what should the response look like if something essential is missing?

#compared to above I just remove "owner"

mini_dataset ={
"ownerGroup":"mygroup",
"accessGroups":["mygroup"],
"creationLocation":"somewhere", 
"principalInvestigator":"someone", 
"datasetName":"test " + datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
"type":"raw",
"isPublished":False,   
"creationTime":str(datetime.now().isoformat()), 
"sourceFolder":"None",
#"owner":"someone",
"ownerEmail":"a@b.de",
"contactEmail":"a@b.de",
"size":0,
"scientificMetadata":{"test":1.0, "test2":{"value":17,"unit":"mA"}},
}


run_all("/api/v3/Datasets",mini_dataset)

====================scicat_new_backend_package====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_new_backend_swap2577====================
{'message': '[{"target":{"ownerGroup":"mygroup","accessGroups":["mygroup"],"creationLocation":"somewhere","principalInvestigator":"someone","datasetName":"test '
            '2022-11-07-14_51_30","type":"raw","isPublished":false,"creationTime":"2022-11-07T14:51:30.080Z","sourceFolder":"None","ownerEmail":"a@b.de","contactEmail":"a@b.de","size":0,"scientificMetadata":{"test":1,"test2":{"value":17,"unit":"mA","valueSI":0.017,"unitSI":"A"}}},"property":"owner","children":[],"constraints":{"isString":"owner '
            'must be a string"}}]',
 'status': 400}




====================scicat_new_backend_master====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_old_backend====================
{'error': {'details': {'codes': {'owner': ['presence'

In [195]:
# what should happen if types are wrong?

mini_dataset ={
"ownerGroup": "mygroup",
"accessGroups":"mygroup",  # this should be a list but is a string on purpose here...
"creationLocation":"somewhere", 
"principalInvestigator":"someone", 
"datasetName":"test " + datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
"type":"raw",
"isPublished":"false",   #should be boolean but is a string on purpose here...
"creationTime":str(datetime.now().isoformat()), 
"sourceFolder":"None",
"owner":"someone",
"ownerEmail":"a@b.de",
"contactEmail":"a@b.de",
"size":0,
"scientificMetadata":{"test":1.0, "test2":{"value":17,"unit":"mA"}},
}

run_all("/api/v3/Datasets",mini_dataset)

====================scicat_new_backend_package====================
{'__v': 0,
 '_id': 'PID.SAMPLE.PREFIXc6b1dd8b-5f24-468a-83c5-2bb95af9fec2',
 'accessGroups': ['mygroup'],
 'attachments': [],
 'contactEmail': 'a@b.de',
 'creationLocation': 'somewhere',
 'creationTime': '2022-11-07T14:51:44.331Z',
 'datablocks': [],
 'datasetName': 'test 2022-11-07-14_51_44',
 'history': [],
 'id': 'PID.SAMPLE.PREFIXc6b1dd8b-5f24-468a-83c5-2bb95af9fec2',
 'isPublished': False,
 'keywords': [],
 'origdatablocks': [],
 'owner': 'someone',
 'ownerEmail': 'a@b.de',
 'ownerGroup': 'mygroup',
 'pid': 'PID.SAMPLE.PREFIXc6b1dd8b-5f24-468a-83c5-2bb95af9fec2',
 'principalInvestigator': 'someone',
 'scientificMetadata': {'test': 1,
                        'test2': {'unit': 'mA',
                                  'unitSI': 'A',
                                  'value': 17,
                                  'valueSI': 0.017}},
 'sharedWith': [],
 'size': 0,
 'sourceFolder': 'None',
 'techniques': [],
 'type': 'raw

In [196]:
#create sample

# is this what a minium sample should look like?

sample={
  "sampleId": "sample1"+ datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
  "owner": "ingestor",
  "description": "DIP on SiOX",
  "sampleCharacteristics": {
    "substate": "SiOx"
  },
  "ownerGroup": "UTUE",
  "accessGroups": [
    "UTUE",
    "ingestor"
  ],
}

run_all("/api/v3/Samples",sample)

# this is the server side message of the new backend package
# MongoServerError: E11000 duplicate key error collection: scicat.Sample index: datasets.pid_1 dup key: { datasets.pid: null }

====================scicat_new_backend_package====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_new_backend_swap2577====================
{'__v': 0,
 '_id': 'sample12022-11-07-14_52_10',
 'accessGroups': ['UTUE', 'ingestor'],
 'description': 'DIP on SiOX',
 'id': 'sample12022-11-07-14_52_10',
 'isPublished': False,
 'owner': 'ingestor',
 'ownerGroup': 'UTUE',
 'sampleCharacteristics': {'substate': 'SiOx'},
 'sampleId': 'sample12022-11-07-14_52_10'}




====================scicat_new_backend_master====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_old_backend====================
{'accessGroups': ['UTUE', 'ingestor'],
 'createdAt': '2022-11-07T13:52:12.002Z',
 'createdBy': 'admin',
 'description': 'DIP on SiOX',
 'owner': 'ingestor',
 'ownerGroup': 'UTUE',
 'sampleCharacteristics': {'substate': 'SiOx'},
 'sampleId': 'sample12022-11-07-14_52_10',
 'updatedAt': '2022-11-07T13:5

In [197]:
# how about adding an 'createdAt' to the sample

sample={
  "sampleId": "sample1"+ datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
  "owner": "ingestor",
  "description": "DIP on SiOX",
  "sampleCharacteristics": {
    "substate": "SiOx"
  },
  "ownerGroup": "UTUE",
  "accessGroups": [
    "UTUE",
    "ingestor"
  ],
  "createdAt": "2021-11-30T09:21:32.054Z", 
}

run_all("/api/v3/Samples",sample)

#was the data model on sample more permissive in the old backend?

====================scicat_new_backend_package====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_new_backend_swap2577====================
{'error': 'Bad Request',
 'message': ['property createdAt should not exist'],
 'statusCode': 400}




====================scicat_new_backend_master====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_old_backend====================
{'accessGroups': ['UTUE', 'ingestor'],
 'createdAt': '2021-11-30T09:21:32.054Z',
 'createdBy': 'admin',
 'description': 'DIP on SiOX',
 'owner': 'ingestor',
 'ownerGroup': 'UTUE',
 'sampleCharacteristics': {'substate': 'SiOx'},
 'sampleId': 'sample12022-11-07-14_52_25',
 'updatedAt': '2022-11-07T13:52:26.553Z'}


In [198]:
#create a proposal

# is this what a minium proposal should look like?

proposal={
  "proposalId": "proposal" + datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
  "email": "l@p.de",
  "title": "study dip on SiOx" + datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
  "abstract": "some text",
  "ownerGroup": "string",
  "accessGroups": [
    "string"
  ],
}

run_all("/api/v3/Proposals",proposal)

#serverside message of new backend package
#  ERROR [ExceptionsHandler] Cannot read properties of undefined (reading 'map')
# TypeError: Cannot read properties of undefined (reading 'map')
#     at updateAllTimesToUTC (/home/node/app/dist/common/utils.js:178:64)
#     at MultiUTCTimeInterceptor.intercept (/home/node/app/dist/common/interceptors/multi-utc-time.interceptor.js:23:67)

====================scicat_new_backend_package====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_new_backend_swap2577====================
{'MeasurementPeriodList': [],
 '__v': 0,
 '_id': 'proposal2022-11-07-14_52_49',
 'abstract': 'some text',
 'accessGroups': ['string'],
 'createdAt': '2022-11-07T13:52:49.713Z',
 'createdBy': 'admin',
 'email': 'l@p.de',
 'id': 'proposal2022-11-07-14_52_49',
 'ownerGroup': 'string',
 'proposalId': 'proposal2022-11-07-14_52_49',
 'title': 'study dip on SiOx2022-11-07-14_52_49',
 'updatedAt': '2022-11-07T13:52:49.713Z',
 'updatedBy': 'admin'}




====================scicat_new_backend_master====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_old_backend====================
{'MeasurementPeriodList': [],
 'abstract': 'some text',
 'accessGroups': ['string'],
 'createdAt': '2022-11-07T13:52:50.412Z',
 'createdBy': 'admin',
 'email': 'l@p.de',
 '

In [199]:
#how about adding updatedAt and createdAt

proposal={
  "proposalId": "proposal" + datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
  "email": "l@p.de",
  "title": "study dip on SiOx" + datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
  "abstract": "some text",
  "ownerGroup": "string",
  "accessGroups": [
    "string"
  ],
  "createdAt": "2021-11-30T09:21:31.402Z",
  "updatedAt": "2021-11-30T09:21:31.402Z",
}

run_all("/api/v3/Proposals",proposal)

#swap 2577 behaves weired here... have a look at response above...

====================scicat_new_backend_package====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_new_backend_swap2577====================
{'error': 'Bad Request',
 'message': ['property createdAt should not exist',
             'property updatedAt should not exist'],
 'statusCode': 400}




====================scicat_new_backend_master====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_old_backend====================
{'MeasurementPeriodList': [],
 'abstract': 'some text',
 'accessGroups': ['string'],
 'createdAt': '2021-11-30T09:21:31.402Z',
 'createdBy': 'admin',
 'email': 'l@p.de',
 'ownerGroup': 'string',
 'proposalId': 'proposal2022-11-07-14_53_07',
 'title': 'study dip on SiOx2022-11-07-14_53_07',
 'updatedAt': '2022-11-07T13:53:09.025Z'}


In [200]:
#create a proposal with MeasurementPeriodList

# is this what a minium proposal should look like?

proposal={
  "proposalId": "proposal" + datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
  "email": "l@p.de",
  "title": "study dip on SiOx" + datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
  "abstract": "some text",
  "ownerGroup": "string",
  "accessGroups": ["string"],
  "MeasurementPeriodList": [
    {
      "id": "string",
      "instrument": "string",
      "start": "2021-11-30T09:21:31.000Z",
      "end": "2021-11-30T09:21:31.000Z",
      "comment": "string",
      "createdBy": "string",
      "updatedBy": "string",
      "createdAt": "2021-11-30T09:21:31.402Z",
      "updatedAt": "2021-11-30T09:21:31.402Z"
    }
  ],
}

run_all("/api/v3/Proposals",proposal)

#serverside message of new backend package
#  ERROR [ExceptionsHandler] E11000 duplicate key error collection: scicat.Proposal index: datasets.pid_1 dup key: { datasets.pid: null }
#  MongoServerError: E11000 duplicate key error collection: scicat.Proposal index: datasets.pid_1 dup key: { datasets.pid: null }

====================scicat_new_backend_package====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_new_backend_swap2577====================
{'error': 'Bad Request',
 'message': ['MeasurementPeriodList.0.an unknown value was passed to the '
             'validate function'],
 'statusCode': 400}




====================scicat_new_backend_master====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_old_backend====================
{'MeasurementPeriodList': [{'comment': 'string',
                            'createdAt': '2021-11-30T09:21:31.402Z',
                            'createdBy': 'string',
                            'end': '2021-11-30T09:21:31.000Z',
                            'id': 'string',
                            'instrument': 'string',
                            'start': '2021-11-30T09:21:31.000Z',
                            'updatedAt': '2021-11-30T09:21:31.402Z',

In [202]:
#create an instrument

instrument = {
#  "pid": "string",
  "name": "string"+ datetime.now().strftime("%Y-%m-%d-%H_%M_%S"),
  "customMetadata": {"test":"test"},
}

run_all("/api/v3/Instruments",instrument)

#server-side error message on new backend package
#ERROR [ExceptionsHandler] E11000 duplicate key error collection: scicat.Instrument index: datasets.pid_1 dup key: { datasets.pid: null }
# MongoServerError: E11000 duplicate key error collection: scicat.Instrument index: datasets.pid_1 dup key: { datasets.pid: null }

====================scicat_new_backend_package====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_new_backend_swap2577====================
{'error': 'Bad Request',
 'message': ['an unknown value was passed to the validate function'],
 'statusCode': 400}




====================scicat_new_backend_master====================
{'message': 'Internal server error', 'statusCode': 500}




====================scicat_old_backend====================
{'createdAt': '2022-11-07T13:55:41.224Z',
 'createdBy': 'admin',
 'customMetadata': {'test': 'test'},
 'name': 'string2022-11-07-14_55_39',
 'pid': 'b4f33ad4-835e-4d10-9dd2-7ed6e78245ee',
 'updatedAt': '2022-11-07T13:55:41.227Z'}
